In [14]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#import dependencies
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = baseline_model()

#Adding Checkpoints to model

filepath="Checkpoints\weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, callbacks=callbacks_list, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))



Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 11s - loss: 0.2791 - acc: 0.9206 - val_loss: 0.1414 - val_acc: 0.9573

Epoch 00001: val_acc improved from -inf to 0.95730, saving model to Checkpoints\weights-01-0.96.hdf5
Epoch 2/10
 - 10s - loss: 0.1117 - acc: 0.9673 - val_loss: 0.0925 - val_acc: 0.9704

Epoch 00002: val_acc improved from 0.95730 to 0.97040, saving model to Checkpoints\weights-02-0.97.hdf5
Epoch 3/10
 - 9s - loss: 0.0718 - acc: 0.9795 - val_loss: 0.0786 - val_acc: 0.9771

Epoch 00003: val_acc improved from 0.97040 to 0.97710, saving model to Checkpoints\weights-03-0.98.hdf5
Epoch 4/10
 - 9s - loss: 0.0501 - acc: 0.9858 - val_loss: 0.0751 - val_acc: 0.9762

Epoch 00004: val_acc did not improve
Epoch 5/10
 - 9s - loss: 0.0373 - acc: 0.9892 - val_loss: 0.0674 - val_acc: 0.9798

Epoch 00005: val_acc improved from 0.97710 to 0.97980, saving model to Checkpoints\weights-05-0.98.hdf5
Epoch 6/10
 - 9s - loss: 0.0269 - acc: 0.9927 - val_loss: 0.0624 - val_acc: 0.

In [13]:
#Cross Validation

from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn = baseline_model, epochs=10, batch_size = 100, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X_test, y_test, cv=10)
print(results.mean())


0.9588000047206879
